In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Getting Started with Vertex AI Turbo Templates

This notebook sets up infrastructure to run production-ready pipelines on Google Cloud. Follow this three-part notebook series to get started in a local Jupyter notebook or in [Vertex AI Workbench](https://cloud.google.com/vertex-ai-notebooks):

1. **[Infrastructure Setup](./02_run_pipelines.ipynb) - this notebook**
1. [Run Pipelines](./02_run_pipelines.ipynb)
1. [Infrastructure Clean Up](./02_run_pipelines.ipynb)


**Prerequisites:**

- [Google Cloud SDK (gcloud)](https://cloud.google.com/sdk/docs/quickstart)
- Make
- [Terraform](https://www.terraform.io)

**For Vertex AI Workbench users**: 
Uncomment and execute the following cell to install Terraform.
Restart the notebook kernel or the Workbench instance to ensure `terraform` is available in the `PATH`.
Then return to this notebook and continue with the next section.

In [1]:
# ! bash ./scripts/install_terraform.sh

./scripts/install_terraform.sh: line 11: terraform: command not found


## Authenticate

Set your project ID and authenticate using your Google Account:

In [1]:
! gcloud config get-value project

Your active configuration is: [thierry-mlops]
uk-gap-proximity-dev


In [2]:
VERTEX_PROJECT_ID = "uk-gap-proximity-dev"
GOOGLE_ACCOUNT = "thierry.dacae@sky.uk"
! gcloud config set project {VERTEX_PROJECT_ID} --quiet
! gcloud config set account {GOOGLE_ACCOUNT} --quiet
! printf 'Y' gcloud auth login

Updated property [core/project].
Updated property [core/account].
Y

## Clone Code

**If you haven't cloned the template, yet:** Uncomment and execute the following cell to clone the code.

In [13]:
# ! git clone -b develop https://github.com/teamdatatonic/vertex-pipelines-end-to-end-samples

Cloning into 'vertex-pipelines-end-to-end-samples'...
remote: Enumerating objects: 4041, done.
remote: Counting objects: 100% (1071/1071), done.
remote: Compressing objects: 100% (462/462), done.
remote: Total 4041 (delta 699), reused 678 (delta 597), pack-reused 2970
Receiving objects: 100% (4041/4041), 3.42 MiB | 24.29 MiB/s, done.
Resolving deltas: 100% (2323/2323), done.


Switch to the folder in which the template code is cloned to:

In [3]:
%cd vertex-pipelines-end-to-end-samples/

/home/jupyter/MLOPS Hackathlon/mlops-hackathon/docs/notebooks/vertex-pipelines-end-to-end-samples


In [4]:
%pwd

'/home/jupyter/MLOPS Hackathlon/mlops-hackathon/docs/notebooks/vertex-pipelines-end-to-end-samples'

Configure your code by setting the variables:
- `VERTEX_PROJECT_ID` - as set above
- `VERTEX_LOCATION` - location of the cloud project
- `BQ_LOCATION` - location of the BigQuery dataset, for this notebook example you can leave this as-is
- `RESOURCE_SUFFIX` - suffix (e.g. `<your name>`) to facilitate running concurrent pipelines in the same Google Cloud project. Change if working in a team to avoid overwriting resources during development 

In [5]:
%%writefile env.sh
#!/bin/bash
VERTEX_PROJECT_ID=VERTEX_PROJECT_ID
VERTEX_LOCATION=europe-west2
BQ_LOCATION=US
RESOURCE_SUFFIX=default

Overwriting env.sh


For most use cases you won't need to change the following variables unless you've modified the Terraform code.

In [6]:
%%writefile -a env.sh
# Optional
VERTEX_CMEK_IDENTIFIER=
VERTEX_NETWORK=
# Leave as-is
VERTEX_SA_EMAIL=vertex-pipelines@${VERTEX_PROJECT_ID}.iam.gserviceaccount.com
VERTEX_PIPELINE_ROOT=gs://${VERTEX_PROJECT_ID}-pl-root
CONTAINER_IMAGE_REGISTRY=${VERTEX_LOCATION}-docker.pkg.dev/${VERTEX_PROJECT_ID}/vertex-images

Appending to env.sh


## Deploy Infrastructure


The cloud infrastructure is managed using Terraform and is defined in the [`terraform`](terraform) directory. There are three Terraform modules defined in [`terraform/modules`](terraform/modules):

- `cloudfunction` - deploys a (Pub/Sub-triggered) Cloud Function from local source code
- `scheduled_pipelines` - deploys Cloud Scheduler jobs that will trigger Vertex Pipeline runs (via the above Cloud Function)
- `vertex_deployment` - deploys Cloud infrastructure required for running Vertex Pipelines, including enabling APIs, creating buckets, Artifact Registry repos, service accounts, and IAM permissions.

**Enable APIs**:

In [7]:
# !gcloud auth login --quiet

# td: run in terminal and copy and paste access code:
gcloud auth login

In [7]:
! gcloud services enable cloudresourcemanager.googleapis.com serviceusage.googleapis.com

Operation "operations/acat.p2-213412891337-f63f28f4-49b0-4e01-8492-b58d194498bc" finished successfully.


In [11]:
VERTEX_PROJECT_ID

'uk-gap-proximity-dev'

In [12]:
VERTEX_LOCATION = "europe-west2"

**Create Cloud Storage bucket:**

Store the [Terraform state files](https://developer.hashicorp.com/terraform/language/state/remote) in the bucket `[project-id]-tfstate`:

In [13]:
! source env.sh && gsutil mb -l $VERTEX_LOCATION -p $VERTEX_PROJECT_ID gs://$VERTEX_PROJECT_ID-tfstate

Creating gs://uk-gap-proximity-dev-tfstate/...


In [16]:
# !terraform init -reconfigure

Terraform initialized in an empty directory!

The directory has no Terraform configuration files. You may begin working
with Terraform immediately by creating Terraform configuration files.


In [19]:
# run in terminal:
# gcloud auth application-default login --project uk-gap-proximity-dev

In [20]:
!gcloud storage ls

gs://artifacts.uk-gap-proximity-dev.appspot.com/
gs://cdn-logs-config-dev/
gs://cdn-suspicious-asn-dev/
gs://cdn_logs_test/
gs://cloud-ai-platform-6e01d309-9443-4e6e-a71b-b118006bee93/
gs://cloud-ai-platform-7f322054-1127-4f3f-a8b6-a013fa86c892/
gs://cloud_logger_test/
gs://dataflow-staging-europe-west1-4b5ff1d7df0f5e4dab4a63d14ebc9c75/
gs://eu.artifacts.uk-gap-proximity-dev.appspot.com/
gs://europe-west1-uk-gap-proximi-11d508c9-bucket/
gs://europe-west1-uk-gap-proximi-4af96600-bucket/
gs://europe-west1-uk-gap-proximi-4ee0444f-bucket/
gs://europe-west1-uk-gap-proximi-586a4af1-bucket/
gs://europe-west1-uk-gap-proximi-65e5278a-bucket/
gs://europe-west1-uk-gap-proximi-9f3b269c-bucket/
gs://europe-west1-uk-gap-proximi-b8fa634c-bucket/
gs://europe-west1-uk-gap-proximi-f5236786-bucket/
gs://europe-west1-uk-gap-proximi-fbf9c379-bucket/
gs://gcf-sources-213412891337-europe-west2/
gs://gcf-v2-sources-213412891337-europe-west1/
gs://gcf-v2-sources-213412891337-europe-west2/
gs://gcf-v2-sources-2

**Deploy:**

In [18]:
! make deploy auto-approve=true

################################################################################
# Deploy dev environment
################################################################################

Initializing the backend...
Initializing modules...
╷
│ Error: Failed to get existing workspaces: querying Cloud Storage failed: storage: bucket doesn't exist
│ 
│ 
╵

make: *** [Makefile:25: deploy] Error 1


You've successfully deployed a `dev` environment! 🎉 
Continue with [this notebook](./02_run_pipelines.ipynb) to run your first Vertex AI Pipelines in the deployed project.

**Note:** If you'd like to deploy separate cloud environments as shown below, try out `make deploy env=dev` where you can replace `dev` with `test` or `prod`.

**Troubleshooting:** If enabling of APIs or the deployment fails, check whether your Google user account has the appropriate permissions.